In [1]:
import pandas as pd
import numpy as np

In [2]:
# files paths
disney_file = "../resources/disney_plus_shows.csv"
netflix_file = "../resources/netflix_titles.csv"
multi_file = "../resources/MoviesOnStreamingPlatforms_updated.csv"

In [3]:
# load into dataframe
df_disney = pd.read_csv(disney_file)
df_netflix = pd.read_csv(netflix_file)
df_multi = pd.read_csv(multi_file)

In [10]:
# clean up disney dataframe
df_disney_clean = df_disney[["imdb_id", "title", "type", "rated", "year", "released_at", "genre", "director", "country"]]

# filter 
df_disney_clean = df_disney_clean.loc[df_disney_clean["type"] == "movie"]
df_disney_clean.columns = ["ID", "Title", "Type", "Rated", "Year", "ReleaseDate", "Genre", "Director", "Country"]
df_disney_clean["ID"] = df_disney_clean["ID"].str.strip("t")
df_disney_clean["ID"] = df_disney_clean["ID"].astype(int)
df_disney_clean = df_disney_clean.reset_index()
df_disney_clean.head()

,index,ID,Title,Type,Rated,Year,ReleaseDate,Genre,Director,Country
0,0,147800,10 Things I Hate About You,movie,PG-13,1999,31 Mar 1999,"Comedy, Drama, Romance",Gil Junger,USA
1,2,115433,101 Dalmatians,movie,G,1996,27 Nov 1996,"Adventure, Comedy, Crime, Family",Stephen Herek,"USA, UK"
2,3,324941,101 Dalmatians 2: Patch's London Adventure,movie,G,2002,21 Jan 2003,"Animation, Adventure, Comedy, Family, Musical","Jim Kammerud, Brian Smith",USA
3,4,211181,102 Dalmatians,movie,G,2000,22 Nov 2000,"Adventure, Comedy, Family",Kevin Lima,"USA, UK"
4,5,1846442,12 Dates of Christmas,movie,PG,2011,11 Dec 2011,"Comedy, Fantasy, Romance",James Hayman,USA


In [7]:
df_netflix.tail()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,"September 25, 2020",2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."
7785,s7786,TV Show,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,"October 31, 2020",2019,TV-PG,1 Season,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...
7786,s7787,Movie,ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS,Sam Dunn,NaN,"United Kingdom, Canada, United States","March 1, 2020",2019,TV-MA,90 min,"Documentaries, Music & Musicals",This documentary delves into the mystique behi...


In [6]:
df_multi.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0
